# testing recordlinkage package
This was ultimately seemingly unsuccessful. The lack of any standards when it comes to wine naming and classification meant that the recordlinkage package struggled to produce usable results since the string distance measures proved inadequate.
Instead, we switched to using the fuzzymatcher provided by github.com/RobinL/fuzzymatcher.

In [16]:
import numpy as np
import json
import pandas as pd
import recordlinkage
import re

In [24]:
vivino = pd.read_csv('/home/p/wines/winerecommender/data-matching/vivino_full.csv', encoding='utf-8')

direktimport = pd.read_csv('/home/p/wines/winerecommender/data-matching/direktimport_formatted.csv', encoding='utf-8')
divino = pd.read_csv('/home/p/wines/winerecommender/data-matching/divino_formatted.csv', encoding='utf-8')
#jacques = pd.read_csv('home/p/wines/winerecommender/data-matching/jacques_wines.csv', encoding='utf-8')

In [25]:
vivino["wine_name"] = vivino["wine_name"].str.lower()
vivino["wine_winery"] = vivino["wine_winery"].str.lower()
direktimport["wine_name"] = direktimport["wine_name"].str.lower()
direktimport["wine_winery"] = direktimport["wine_winery"].str.lower()


In [51]:
indexer = recordlinkage.Index()
indexer.block(left_on='wine_type', right_on='wine_type')
indexer.block(left_on='wine_country', right_on='wine_country')

candidate_pairs = indexer.index(vivino, direktimport)

In [52]:
compare_wines = recordlinkage.Compare()
compare_wines.string('wine_name', 'wine_name', method='jarowinkler', threshold=0.8, label='name')
compare_wines.string('wine_winery', 'wine_winery', method='jarowinkler', threshold=0.80, label='winery')
compare_wines.numeric('wine_alcohol', 'wine_alcohol', label = 'alc')
compare_wines.string('wine_region', 'wine_region', method='jarowinkler', threshold=0.80, label='region')

<Compare>

In [53]:
X = compare_wines.compute(candidate_pairs, vivino, direktimport)
res = X[X.sum(axis=1) > 3]
res.loc[0]

# test recommendation function

In [70]:
with open('/home/p/wines/winerecommender/WineRecommender_API/recommender_engine/example_json/example_input_profile.json') as f:
    profile = json.load(f)

with open('/home/p/wines/winerecommender/WineRecommender_API/recommender_engine/example_json/example_input_wines.json') as f:
    wines = json.load(f)

In [71]:
def calcRecs(wines_list, user_profile, num_wines = 10, structure_param = 0.5, tastes_param = 1, ratings_param = 1):
    user_structure = np.asarray(list(user_profile["structure"].values()))
    user_tastes = np.asarray(list(user_profile["tastes"].values()))

    for wine in wines_list:
        wine_tastes = np.asarray(list(wine["wine_tastes"].values()))
        wine_structure = np.asarray(list(wine["wine_structure"].values()))
        wine["score"] = (structure_param * np.sum(np.multiply(user_structure, wine_structure)) + tastes_param * np.sum(np.multiply(user_tastes, wine_tastes))) * (ratings_param * float(wine["wine_rating"])/4)
    
    wines_list = sorted(wines_list, key=lambda k: k["score"], reverse=True)

    if num_wines > len(wines_list):
        num_wines = len(wines_list)

    return wines_list[:num_wines-1]


In [72]:
print(calcRecs(wines, profile))



[{'wine_id': 77658, 'wine_structure': {'acidity': 0.3651303, 'fizziness': 0, 'intensity': 0.7317925, 'sweetness': 0.4346559, 'tannin': 0}, 'wine_tastes': {'black_fruit': 0.011428571428571429, 'citrus_fruit': 0.2742857142857143, 'dried_fruit': 0.011428571428571429, 'earth': 0.49142857142857144, 'floral': 0.17142857142857143, 'microbio': 0.14857142857142858, 'non_oak': 0.04, 'oak': 0.09142857142857143, 'red_fruit': 0.005714285714285714, 'spices': 0.045714285714285714, 'tree_fruit': 0.5657142857142857, 'tropical_fruit': 1.0, 'vegetal': 0.10857142857142857}, 'wine_rating': 3.9, 'score': 2.087711202420718}, {'wine_id': 73456, 'wine_structure': {'acidity': 0.70047925, 'fizziness': 0, 'intensity': 0.8207311749999999, 'sweetness': 0.14430955, 'tannin': 0.726551425}, 'wine_tastes': {'black_fruit': 0.7629179331306991, 'citrus_fruit': 0.0243161094224924, 'dried_fruit': 0.03343465045592705, 'earth': 0.601823708206687, 'floral': 0.03951367781155015, 'microbio': 0.03951367781155015, 'non_oak': 0.021

some data exploration of keys

In [ ]:
#exploration for wine region matching between vivino and local data

def getRegionSet(array):
    seen = set()
    regions = []
    for region in array:
        region = str(region).strip().split(', ')[0]
        if region not in seen:
            seen.add(region)
            regions.append(region)
    return sorted(regions)

regions = getRegionSet(final['wine_region'])

vivreg = getRegionSet([wine['wine_region'] for wine in vivino_data])

print('Matching: ',[region for region in regions if region in vivreg])
print('Missing: ',[region for region in regions if region not in vivreg])

In [7]:
import json
with open('/home/p/wines/winerecommender/data-matching/vivino_data_combined_fm.json') as f:
    viv = json.load(f)
seen = set()
keys = []
for wine in viv:
    try:
        for group in wine["wine_tastes"]:
            if group["group"] not in seen:
                seen.add(group["group"])
                keys.append(group["group"])
    except:
        continue
print(sorted(keys))

['black_fruit', 'citrus_fruit', 'dried_fruit', 'earth', 'floral', 'microbio', 'non_oak', 'oak', 'red_fruit', 'spices', 'tree_fruit', 'tropical_fruit', 'vegetal']


In [1]:
import json
with open('/home/p/wines/winerecommender/data-matching/vivino_data_combined_fm.json') as f:
    viv = json.load(f)
seen = set()
keys = []
for wine in viv:
    try:
        x = wine["wine_country"]
        if x not in seen:
            seen.add(x)
            keys.append(x)
    except:
        continue
print(sorted(keys))

['Argentinien', 'Armenien', 'Australien', 'Austria', 'Belgien', 'Brasilien', 'Bulgarien', 'Chile', 'Czech Republic', 'Deutschland', 'France', 'Frankreich', 'Georgien', 'Germany', 'Griechenland', 'Israel', 'Italien', 'Italy', 'Kanada', 'Kroatien', 'Libanon', 'Luxemburg', 'Mexiko', 'Neuseeland', 'Peru', 'Portugal', 'Republik Moldau', 'Rumänien', 'Schweiz', 'Serbia', 'Slowenien', 'Spain', 'Spanien', 'Switzerland', 'Südafrika', 'Tschechische Republik', 'Türkei', 'Ungarn', 'United States', 'Uruguay', 'Vereinigte Staaten', 'Vereinigtes Königreich', 'Zypern', 'Österreich']


testing user profile generation

In [ ]:
import json
with open('/home/p/wines/winerecommender/WineRecommender_API/recommender_engine/example_json/wine_sample.json') as f:
    test_wines = json.load(f)

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)
test_obj = []
for wine in test_wines:
    test_obj.append(Struct(**wine))


template = {

    "taste_data": {
        "black_fruit": 0,
        "citrus_fruit": 0,
        "dried_fruit": 0,
        "earth": 0,
        "floral": 0,
        "microbio": 0,
        "non_oak": 0,
        "oak": 0,
        "red_fruit": 0,
        "spices": 0,
        "tree_fruit": 0,
        "tropical_fruit": 0,
        "vegetal": 0
    },
    "structure_data": {
        "acidity": 0,
        "fizziness": 0,
        "intensity": 0,
        "sweetness": 0,
        "tannin": 0
        }
}
#assumes input to be a list of wines, as well as template to fill in
def generateProfile(wines, template):
    
    keys_taste = ['black_fruit', 'citrus_fruit', 'dried_fruit', 'earth', 'floral', 'microbio', 'non_oak', 'oak', 'red_fruit', 'spices', 'tree_fruit', 'tropical_fruit', 'vegetal']
    keys_structure = ["acidity", "fizziness", "intensity", "sweetness", "tannin"]
    # types_all = ["red", "white", "sparkling", "rosé"]
   
    # average_taste = np.zeros(13)
    # average_structure = np.zeros(5)
    seen_high = False
    seen_medium = False
    seen_low = False
    origins = set()
    types = set()

    for wine in wines:
        if wine.wine_country not in origins:
            origins.add(wine.wine_region)
        if wine.wine_type not in types:
            types.add(wine.wine_type)
        if not seen_low and wine.price < 10 :
            options[0]["selected"] = True
        if not seen_medium and wine.price > 10 and wine.price < 20:
            options[1]["selected"] = True
        if not seen_high and wine.price > 20:
            options[2]["selected"] = True
        
        for key in keys_taste:
            template["taste_data"][key] += wine.wine_tastes[key]
        for key in keys_structure:
            template["structure_data"][key] += wine.wine_structure[key]
        
    for key in keys_taste:
        template["taste_data"][key] = template["taste_data"][key] / len(wines)
    for key in keys_structure:
        template["structure_data"][key] = template["structure_data"][key] / len(wines)
    
    return template, types, origins, (seen_low, seen_medium, seen_high)

In [23]:
import json
profile = {
    "wine_data": [
        {
            "selection_type": "multiselect",
            "name": "Type of Wine",
            "options": [
                {
                    "option": "red",
                    "selected": True
                },
                {
                    "option": "sparkling",
                    "selected": False
                },
                {
                    "option": "white",
                    "selected": False
                },
                {
                    "option": "rosé",
                    "selected": False
                }
            ]
        },
        {
            "selection_type": "multiselect",
            "name": "Price",
            "options": [
                {
                    "option": "under 10€",
                    "selected": True
                },
                {
                    "option": "10-20€",
                    "selected": False
                },
                {
                    "option": "over 20€",
                    "selected": False
                }
            ]
        },
        {
            "selection_type": "search_field",
            "name": "Origin",
            "options": [
                {
                    "option": "Mexiko",
                    "selected": True
                },
                {
                    "option": "Italy",
                    "selected": False
                },
                {
                    "option": "Neuseeland",
                    "selected": False
                },
                {
                    "option": "Uruguay",
                    "selected": False
                },
                {
                    "option": "Deutschland",
                    "selected": False
                },
                {
                    "option": "Vereinigtes Königreich",
                    "selected": False
                },
                {
                    "option": "Frankreich",
                    "selected": False
                },
                {
                    "option": "Italien",
                    "selected": True
                },
                {
                    "option": "Austria",
                    "selected": False
                },
                {
                    "option": "Spanien",
                    "selected": False
                },
                {
                    "option": "Australia",
                    "selected": False
                },
                {
                    "option": "Germany",
                    "selected": False
                },
                {
                    "option": "Luxemburg",
                    "selected": False
                },
                {
                    "option": "Argentinien",
                    "selected": False
                },
                {
                    "option": "Portugal",
                    "selected": False
                },
                {
                    "option": "Griechenland",
                    "selected": False
                },
                {
                    "option": "Argentina",
                    "selected": False
                },
                {
                    "option": "Spain",
                    "selected": False
                },
                {
                    "option": "Südafrika",
                    "selected": False
                },
                {
                    "option": "Chile",
                    "selected": False
                },
                {
                    "option": "France",
                    "selected": False
                },
                {
                    "option": "Österreich",
                    "selected": False
                },
                {
                    "option": "Vereinigte Staaten",
                    "selected": False
                },
                {
                    "option": "Brasilien",
                    "selected": False
                },
                {
                    "option": "Israel",
                    "selected": False
                },
                {
                    "option": "South Africa",
                    "selected": False
                },
                {
                    "option": "Australien",
                    "selected": False
                },
                {
                    "option": "Schweiz",
                    "selected": False
                },
                {
                    "option": "United States",
                    "selected": False
                }
            ]
        }
    ],
    "taste_data": [
        {
            "label": "black_fruit",
            "percentage": 0.5236220472440944
        },
        {
            "label": "citrus_fruit",
            "percentage": 0.007874015748031496
        },
        {
            "label": "dried_fruit",
            "percentage": 0.03937007874015748
        },
        {
            "label": "earth",
            "percentage": 0.14960629921259844
        },
        {
            "label": "floral",
            "percentage": 0.011811023622047244
        },
        {
            "label": "microbio",
            "percentage": 0.07480314960629922
        },
        {
            "label": "non_oak",
            "percentage": 0.027559055118110236
        },
        {
            "label": "oak",
            "percentage": 1
        },
        {
            "label": "red_fruit",
            "percentage": 0.40551181102362205
        },
        {
            "label": "spices",
            "percentage": 0.1968503937007874
        },
        {
            "label": "tree_fruit",
            "percentage": 0.003937007874015748
        },
        {
            "label": "tropical_fruit",
            "percentage": 0
        },
        {
            "label": "vegetal",
            "percentage": 0.003937007874015748
        }
    ]
}





In [26]:
import numpy as np
keys_taste = ['black_fruit', 'citrus_fruit', 'dried_fruit', 'earth', 'floral', 'microbio', 'non_oak', 'oak',
              'red_fruit', 'spices', 'tree_fruit', 'tropical_fruit', 'vegetal']
wine_data = profile["wine_data"]
taste_data = profile["taste_data"]
types = [option["option"] for option in wine_data[0]["options"] if option["selected"] == True]
origins = [option["option"] for option in wine_data[2]["options"] if option["selected"] == True]
ranges = [option["option"] for option in wine_data[1]["options"] if option["selected"] == True]
user_taste_dict = {taste["label"]: taste["percentage"] for taste in taste_data}
user_taste = np.asarray([user_taste_dict[key] for key in keys_taste])


print(types, origins, ranges, user_taste)

['red'] ['Mexiko', 'Italien'] ['under 10€'] [0.52362205 0.00787402 0.03937008 0.1496063  0.01181102 0.07480315
 0.02755906 1.         0.40551181 0.19685039 0.00393701 0.
 0.00393701]


In [28]:
test = [1,2,3]

for i in range(len(test)):
    print(i)

0
1
2
